### Подготовка: установка зависимостей, импорты и загрузка/выгрузка внешних переменных

In [1]:
!pip install -r requirements.txt

In [2]:
import requests
import os
import json
import langchain

%reload_ext dotenv
%dotenv

api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

### Класс для взаимодействия с моделью - заполняет поля, отправляет запрос и получает ответ

In [3]:
from typing import Optional, List, Mapping, Any
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

class YandexLLM(LLM):
    api_key: str = None
    folder_id: str = None
    max_tokens : int = 1000
    temperature : float = 0.6
    instruction_text : str = None
    prompt: str = None
        
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        headers = {'Authorization': f'Api-Key {self.api_key}', "x-folder-id": f"{self.folder_id}"}
        req_completion = {
            "modelUri": f"gpt://{self.folder_id}/yandexgpt-lite/latest",
            "completionOptions": {
                "temperature": self.temperature,
                "maxTokens": self.max_tokens 
            },
            "messages": [
                {
                    "role": "system",
                    "text": "Представь, что ты сотрудник компании MyApp. Отвечай на вопросы вежливо и по мере своих возможностей."
                }
            ]
        }
        # req_instruct = {
        #     "model": "general",
        #     "instruction_text": self.instruction_text,
        #     "request_text": self.prompt,
        #     "generation_options": {
        #         "max_tokens": self.max_tokens,
        #         "temperature": self.temperature
        #     }
        #     #"requestText": "Привет!"
        # }

        print("Sending request to TextGeneration.instruct API")

        res = requests.post(url="https://llm.api.cloud.yandex.net/foundationModels/v1/completion", #("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
                            headers=headers, json=req_completion)
        
        print("Received response from Yandex LLM API.")
        
        return res.text

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"model_name": "YandexGPT_RAGbot"}
    
    @property
    def _llm_type(self) -> str:
        return "yagpt"


### Подготовка переменных для промпта

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./documents/reg_account.txt")
input_doc = loader.load()

query = "Куда должно прийти письмо с подтверждением регистрации?"

### Создание промпта и цепочки для LLM

In [5]:
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Instruction for YandexGPT
# instruction = "Представь, что ты сотрудник компании MyApp. Отвечай на вопросы вежливо и по мере своих возможностей."

# It accepts a set of parameters from the user that can be used to generate a prompt 
# Example for understanding:
# prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}") 
# If foo="Hello", Result: "Say Hello"
document_template = PromptTemplate(
    input_variables=["page_content"], #A list of the names of the variables, that document prompt template expect (as a variable type).
    template="{page_content}" # The document prompt template.
)

# The variable name where to put document in final prompt.
document_variable_name = "context"

# Template for prompt
template_override = """
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""
# create prompt
prompt = langchain.prompts.PromptTemplate(
    input_variables=["context", "query"],
    template=template_override
)

# LLM object 
llm = YandexLLM(api_key=api_key, folder_id=folder_id)

# create chain step 1: llm and prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# create chain step 2: llm, prompt. And add: document, document variable 
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_template,
    document_variable_name=document_variable_name
)

In [6]:
result = chain.run(input_documents=input_doc, query=query)
print(result)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sending request to TextGeneration.instruct API
Received response from Yandex LLM API.
{"result":{"alternatives":[{"message":{"role":"assistant","text":"Я, как сотрудник компании MyApp, рад помочь! Вот мои ответы на ваши вопросы:\n\n**Как установить приложение MyApp?**\n\nДля установки приложения MyApp вам необходимо найти его в соответствующем магазине приложений или на официальном сайте и установить его на ваше устройство. Процесс установки может варьироваться в зависимости от операционной системы вашего устройства, но обычно это довольно простой процесс, требующий лишь нескольких нажатий.\n\n**Какие устройства поддерживает приложение MyApp?**\n\nПриложение MyApp поддерживает широкий спектр устройств, включая смартфоны и планшеты с различными операционными системами, такими как iOS, Android, Windows Phone и другими. Оно также может быть адаптировано для работы на различных устройствах с помощью веб-версии, если это необходимо.\n\n**Какова цель приложения MyApp?**\n\nОсновная цель прил

In [ ]:
result = chain.run(input_documents=input_doc, query=query)
print(result)